In [1]:
import json
import os
from sys import argv
from typing import Dict, List

from tqdm import tqdm

from allennlp.common.util import import_submodules
from allennlp.data import DataIterator, DatasetReader
from allennlp.data.dataset import Batch
from allennlp.models.archival import load_archive
from allennlp.nn import util as nn_util

import logging
logging.basicConfig(format="%(asctime)s:%(levelname)s:%(message)s", level=logging.INFO)

/cluster/home/fgonzalez/.virtualenvs/scirex/lib64/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
/cluster/home/fgonzalez/.virtualenvs/scirex/lib64/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated

In [2]:

archive_folder = "/cluster/scratch/fgonzalez/SciREX/outputs/pwc_outputs/experiment_scirex_full/main3"
test_file = "/cluster/scratch/fgonzalez/test_outputs/ner_predictions.jsonl"
output_file = "/cluster/scratch/fgonzalez/test_outputs/salient_mentions_predictions.jsonl"
cuda_device = int(0)


In [8]:
!export PYTHONPATH="/cluster/scratch/fgonzalez/SciREX"

In [3]:
import sys
sys.path.append("/cluster/scratch/fgonzalez/SciREX")

In [4]:
import_submodules("scirex")

/cluster/home/fgonzalez/.virtualenvs/scirex/lib64/python3.7/site-packages/sklearn/decomposition/online_lda.py:29: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [5]:

logging.info("Loading Model from %s", archive_folder)
archive_file = os.path.join(archive_folder, "model.tar.gz")
archive = load_archive(archive_file, cuda_device)
model = archive.model
model.eval()

saliency_threshold = json.load(open(archive_folder + '/metrics.json'))['best_validation__span_threshold']

model.prediction_mode = True
config = archive.config.duplicate()
dataset_reader_params = config["dataset_reader"]
dataset_reader = DatasetReader.from_params(dataset_reader_params)
dataset_reader.prediction_mode = True
instances = dataset_reader.read(test_file)


2022-03-16 18:20:36,281:INFO:Loading Model from /cluster/scratch/fgonzalez/SciREX/outputs/pwc_outputs/experiment_scirex_full/main3
2022-03-16 18:20:36,285:INFO:loading archive file /cluster/scratch/fgonzalez/SciREX/outputs/pwc_outputs/experiment_scirex_full/main3/model.tar.gz
2022-03-16 18:20:36,286:INFO:extracting archive file /cluster/scratch/fgonzalez/SciREX/outputs/pwc_outputs/experiment_scirex_full/main3/model.tar.gz to temp dir /scratch/209030495.tmpdir/tmpdi0xd90s
2022-03-16 18:20:39,263:INFO:instantiating registered subclass scirex_model of <class 'allennlp.models.model.Model'>
2022-03-16 18:20:39,264:INFO:type = default
2022-03-16 18:20:39,264:INFO:instantiating registered subclass default of <class 'allennlp.data.vocabulary.Vocabulary'>
2022-03-16 18:20:39,265:INFO:Loading token dictionary from /scratch/209030495.tmpdir/tmpdi0xd90s/vocabulary.
2022-03-16 18:20:39,266:INFO:instantiating class <class 'allennlp.models.model.Model'> from params {'context_layer': {'bidirectional':

2022-03-16 18:20:43,989:INFO:instantiating class <class 'allennlp.modules.feedforward.FeedForward'> from params {'activations': 'gelu', 'dropout': 0.2, 'hidden_dims': 150, 'input_dim': 400, 'num_layers': 2} and extras {'vocab'}
2022-03-16 18:20:43,989:INFO:ner.mention_feedforward.input_dim = 400
2022-03-16 18:20:43,990:INFO:ner.mention_feedforward.num_layers = 2
2022-03-16 18:20:43,990:INFO:ner.mention_feedforward.hidden_dims = 150
2022-03-16 18:20:43,990:INFO:ner.mention_feedforward.activations = gelu
2022-03-16 18:20:43,990:INFO:instantiating registered subclass gelu of <class 'allennlp.nn.activations.Activation'>
2022-03-16 18:20:43,991:INFO:ner.mention_feedforward.dropout = 0.2
2022-03-16 18:20:43,993:INFO:ner.label_namespace = ner_type_labels
2022-03-16 18:20:43,993:INFO:ner.label_encoding = BIOUL
2022-03-16 18:20:43,993:INFO:ner.exact_match = false
2022-03-16 18:20:43,996:INFO:Initializing parameters
2022-03-16 18:20:43,996:INFO:Done initializing parameters; the following paramet

2022-03-16 18:20:44,028:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.0
2022-03-16 18:20:44,029:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.1
2022-03-16 18:20:44,029:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.10
2022-03-16 18:20:44,029:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.11
2022-03-16 18:20:44,029:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.2
2022-03-16 18:20:44,030:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.3
2022-03-16 18:20:44,030:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.4
2022-03-16 18:20:44,030:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.5
2022-03-16 18:20:44,031:INFO:   _text_field_embedder.token_embedder_bert._scalar_mix.scalar_parameters.6
2022-03-16 18:20:44,031:INFO:   _text_field_embedder.

2022-03-16 18:20:44,053:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.output.dense.weight
2022-03-16 18:20:44,053:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.self.key.bias
2022-03-16 18:20:44,054:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.self.key.weight
2022-03-16 18:20:44,054:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.self.query.bias
2022-03-16 18:20:44,054:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.self.query.weight
2022-03-16 18:20:44,055:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.self.value.bias
2022-03-16 18:20:44,055:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.attention.self.value.weight
2022-03-16 18:20:44,055:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.11.inte

2022-03-16 18:20:44,071:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.attention.self.key.bias
2022-03-16 18:20:44,071:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.attention.self.key.weight
2022-03-16 18:20:44,071:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.attention.self.query.bias
2022-03-16 18:20:44,072:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.attention.self.query.weight
2022-03-16 18:20:44,072:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.attention.self.value.bias
2022-03-16 18:20:44,072:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.attention.self.value.weight
2022-03-16 18:20:44,072:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.intermediate.dense.bias
2022-03-16 18:20:44,073:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.5.intermediate.dense

2022-03-16 18:20:44,089:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.attention.self.query.bias
2022-03-16 18:20:44,089:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.attention.self.query.weight
2022-03-16 18:20:44,089:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.attention.self.value.bias
2022-03-16 18:20:44,089:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.attention.self.value.weight
2022-03-16 18:20:44,090:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.intermediate.dense.bias
2022-03-16 18:20:44,090:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.intermediate.dense.weight
2022-03-16 18:20:44,090:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.output.LayerNorm.bias
2022-03-16 18:20:44,091:INFO:   _text_field_embedder.token_embedder_bert.bert_model.encoder.layer.9.output.LayerNorm.wei

{0: 'O', 1: 'B-Method', 2: 'L-Method', 3: 'I-Method', 4: 'U-Method', 5: 'B-Task', 6: 'L-Task', 7: 'I-Task', 8: 'B-Metric', 9: 'L-Metric', 10: 'U-Task', 11: 'U-Metric', 12: 'B-Material', 13: 'L-Material', 14: 'I-Metric', 15: 'I-Material', 16: 'U-Material'}


2022-03-16 18:20:47,274:INFO:instantiating class <class 'allennlp.data.dataset_readers.dataset_reader.DatasetReader'> from params {'token_indexers': {'bert': {'do_lowercase': 'true', 'pretrained_model': '/cluster/scratch/fgonzalez/scibert/scibert_scivocab_uncased//vocab.txt', 'truncate_long_sequences': False, 'type': 'bert-pretrained', 'use_starting_offsets': True}}, 'type': 'scirex_full_reader'} and extras set()
2022-03-16 18:20:47,274:INFO:dataset_reader.type = scirex_full_reader
2022-03-16 18:20:47,275:INFO:instantiating class <class 'scirex.data.dataset_readers.scirex_full_reader.ScirexFullReader'> from params {'token_indexers': {'bert': {'do_lowercase': 'true', 'pretrained_model': '/cluster/scratch/fgonzalez/scibert/scibert_scivocab_uncased//vocab.txt', 'truncate_long_sequences': False, 'type': 'bert-pretrained', 'use_starting_offsets': True}}} and extras set()
2022-03-16 18:20:47,275:INFO:instantiating class <class 'allennlp.data.token_indexers.token_indexer.TokenIndexer'> from p

In [6]:
for instance in instances :
    batch = Batch([instance])
    batch.index_instances(model.vocab)



In [7]:
data_iterator = DataIterator.from_params(config["validation_iterator"])
iterator = data_iterator(instances, num_epochs=1, shuffle=False)

2022-03-16 18:20:51,096:INFO:instantiating class <class 'allennlp.data.iterators.data_iterator.DataIterator'> from params {'batch_size': 10, 'type': 'ie_batch'} and extras set()
2022-03-16 18:20:51,097:INFO:validation_iterator.type = ie_batch
2022-03-16 18:20:51,098:INFO:instantiating class <class 'scirex.data.iterators.batch_iterator.BatchIterator'> from params {'batch_size': 10} and extras set()
2022-03-16 18:20:51,098:INFO:validation_iterator.batch_size = 10
2022-03-16 18:20:51,098:INFO:validation_iterator.instances_per_epoch = None
2022-03-16 18:20:51,099:INFO:validation_iterator.max_instances_in_memory = None
2022-03-16 18:20:51,099:INFO:validation_iterator.cache_instances = False
2022-03-16 18:20:51,099:INFO:validation_iterator.track_epoch = False
2022-03-16 18:20:51,100:INFO:validation_iterator.maximum_samples_per_batch = None


In [88]:
instance.fields

{'text': <allennlp.data.fields.text_field.TextField at 0x2adc4ad34510>,
 'ner_type_labels': <allennlp.data.fields.sequence_label_field.SequenceLabelField at 0x2adc4ad345d0>,
 'metadata': <allennlp.data.fields.metadata_field.MetadataField at 0x2adc4ad34550>,
 'spans': <allennlp.data.fields.list_field.ListField at 0x2adc4ad34810>,
 'span_cluster_labels': <allennlp.data.fields.list_field.ListField at 0x2adc4ad346d0>,
 'span_saliency_labels': <allennlp.data.fields.sequence_label_field.SequenceLabelField at 0x2adc4ad34610>,
 'span_type_labels': <allennlp.data.fields.sequence_label_field.SequenceLabelField at 0x2adc4ad34650>,
 'span_features': <allennlp.data.fields.list_field.ListField at 0x2adc4ad34850>}

In [78]:
instance.get('spans').as_tensor

<bound method ListField.as_tensor of <allennlp.data.fields.list_field.ListField object at 0x2adc4ad34810>>

In [66]:
for batch in tqdm(iterator):
    print(batch["spans"])

44it [00:00, 218.38it/s]

tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1,

88it [00:00, 214.18it/s]

tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1,

102it [00:00, 211.22it/s]

tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]])
tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1,

In [65]:
batch["spans"]

tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]], device='cuda:0')

In [ ]:
def decode_saliency(self, batch, saliency_threshold):
    output_embedding = self.embedding_forward(text=batch["text"])
    output_span_embedding = self.span_embeddings_forward(
        output_embedding=output_embedding,
        spans=batch["spans"],
        span_type_labels=batch["span_type_labels"],
        span_features=batch["span_features"],
        metadata=batch["metadata"],
    )

    output_saliency = self.saliency_forward(
        output_span_embedding=output_span_embedding,
        metadata=batch["metadata"],
        span_saliency_labels=batch["span_saliency_labels"],
        span_cluster_labels=batch["span_cluster_labels"],
        saliency_threshold=saliency_threshold,
    )

    return self._saliency_classifier.decode(output_saliency)

In [38]:
output_embedding = model.embedding_forward(text=batch["text"])

In [39]:
text=batch["text"]

In [42]:
from allennlp.nn import InitializerApplicator, RegularizerApplicator, util

In [43]:
text_embeddings = model._lexical_dropout(model._text_field_embedder(text))
text_mask = util.get_text_field_mask(text)
sentence_lengths = text_mask.sum(-1)

In [50]:
flat_text_embeddings = text_embeddings.view(-1, text_embeddings.size(-1))
flat_text_mask = text_mask.view(-1).byte()

In [53]:
filtered_text_embeddings = flat_text_embeddings[flat_text_mask.bool()]

In [57]:
import torch

In [190]:
doc_ids: List[str]= []

In [58]:
filtered_contextualized_embeddings = model._context_layer(
        filtered_text_embeddings.unsqueeze(0),
        torch.ones((1, filtered_text_embeddings.size(0)), device=filtered_text_embeddings.device).byte(),
    ).squeeze(0)

In [60]:
filtered_text_embeddings

tensor([[-0.7595,  0.1845, -0.5993,  ..., -0.0827, -0.9913, -0.5960],
        [ 0.4092, -0.5473,  0.1391,  ..., -0.2535, -1.3709, -0.8746],
        [ 0.0253,  0.2964, -0.2465,  ..., -0.5693, -0.5737, -0.0314],
        ...,
        [-0.8255,  0.2050, -0.5399,  ..., -1.0783, -1.0213,  0.0763],
        [-0.0963,  0.2643, -0.3277,  ..., -0.3970, -0.4930, -0.6844],
        [-0.7601, -0.5484, -0.5534,  ..., -0.7049, -1.4994,  0.5445]],
       device='cuda:0', grad_fn=<IndexBackward>)

In [59]:
filtered_contextualized_embeddings

tensor([[-4.6825e-04, -1.2295e-02,  1.1463e-02,  ...,  2.6295e-03,
         -1.7944e-03,  7.7497e-03],
        [-1.7461e-03, -6.1071e-01,  1.5967e-02,  ...,  3.0832e-02,
          2.8066e-03,  6.0617e-01],
        [ 1.4560e-01, -5.8072e-04,  3.9946e-07,  ..., -6.8800e-04,
          7.6671e-04,  3.3265e-01],
        ...,
        [-4.8250e-03, -8.5825e-01,  1.2302e-01,  ...,  1.8657e-02,
         -1.3855e-04,  5.2764e-01],
        [-1.3995e-02, -9.2483e-01,  3.1442e-01,  ...,  3.5080e-01,
          1.0244e-02,  5.9603e-01],
        [-2.0613e-01, -4.5218e-01,  5.9343e-02,  ...,  2.7646e-01,
          4.2228e-04,  9.3150e-02]], device='cuda:0',
       grad_fn=<SqueezeBackward1>)

In [ ]:
def embedding_forward(self, text):
    # Shape: (batch_size, max_sentence_length, embedding_size)
    

    # Shape: (total_sentence_length, encoding_dim)
    

    
    

    flat_contextualized_embeddings = torch.zeros(
        (flat_text_embeddings.size(0), filtered_contextualized_embeddings.size(1)),
        device=filtered_text_embeddings.device,
    )
    flat_contextualized_embeddings.masked_scatter_(
        flat_text_mask.unsqueeze(-1).bool(), filtered_contextualized_embeddings
    )

    # Shape: (batch_size, max_sentence_length, embedding_size)
    contextualized_embeddings = flat_contextualized_embeddings.reshape(
        (text_embeddings.size(0), text_embeddings.size(1), flat_contextualized_embeddings.size(-1))
    )

    output_embedding = {
        "contextualised": contextualized_embeddings,
        "text": text_embeddings,
        "mask": text_mask,
        "lengths": sentence_lengths,
    }
    return output_embedding

In [21]:
output_span_embedding = model.span_embeddings_forward(
        output_embedding=output_embedding,
        spans=batch["spans"],
        span_type_labels=batch["span_type_labels"],
        span_features=batch["span_features"],
        metadata=batch["metadata"],
    )

In [25]:
output_span_embeddings = {"valid": False}

In [63]:
output_embedding=output_embedding
spans=batch["spans"]
span_type_labels=batch["span_type_labels"]
span_features=batch["span_features"]
metadata=batch["metadata"]

In [31]:
spans.nelement()

10

In [33]:
span_mask, spans, span_embeddings = model.extract_span_embeddings(
                output_embedding["contextualised"], spans
            )

In [64]:
spans

tensor([[[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]],

        [[-1, -1]]], device='cuda:0')

In [37]:
span_mask

tensor([[0],
        [0],
        [0],
        [0],
        [0]], device='cuda:0')

In [15]:
 def span_embeddings_forward(self, output_embedding, spans, span_type_labels, span_features, metadata):
        

        if spans.nelement() != 0:
            span_mask, spans, span_embeddings = self.extract_span_embeddings(
                output_embedding["contextualised"], spans
            )

            if span_mask.sum() != 0:
                span_offset = self.offset_span_by_para_start(metadata, spans, span_mask)
                span_position = self.get_span_position(metadata, span_offset)
                span_type_labels_one_hot = self.get_span_one_hot_labels(
                    "span_type_labels", span_type_labels, spans
                )

                span_features = torch.cat(
                    [span_position, span_type_labels_one_hot, span_features.float()], dim=-1
                )
                featured_span_embeddings = torch.cat([span_embeddings, span_features], dim=-1)
                span_ix = span_mask.view(-1).nonzero().squeeze(1).long()

                output_span_embeddings = {
                    "span_mask": span_mask,
                    "span_ix": span_ix,
                    "spans": span_offset,
                    "span_embeddings": span_embeddings,
                    "featured_span_embeddings": featured_span_embeddings,
                    "span_type_labels": span_type_labels_one_hot,
                    "span_features": span_features.float(),
                    "valid": True,
                }

        return output_span_embeddings

{'text': {'bert': tensor([[  102, 13271,   862,  ...,  9996,   205,   103],
          [  102, 13271,   862,  ...,     0,     0,     0],
          [  102,  1155,   862,  ...,     0,     0,     0],
          [  102,  1155,   862,  ...,     0,     0,     0],
          [  102, 29098,   862,  ...,     0,     0,     0]], device='cuda:0'),
  'bert-offsets': tensor([[  1,   2,   3,  ..., 303, 304, 305],
          [  1,   2,   3,  ...,   0,   0,   0],
          [  1,   2,   3,  ...,   0,   0,   0],
          [  1,   2,   3,  ...,   0,   0,   0],
          [  1,   2,   3,  ...,   0,   0,   0]], device='cuda:0'),
  'bert-type-ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]], device='cuda:0'),
  'mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  ..., 0, 0, 0],
          [1, 1, 1,  .

In [16]:
saliency_threshold

0.2631010101010101

In [14]:
for batch in tqdm(iterator):
    #print(batch.keys())
    batch = nn_util.move_to_device(batch, cuda_device)  # Put on GPU.
    print(batch['metadata'][0]['doc_id'])
    output_res = model.decode_saliency(batch, saliency_threshold)
    print(output_res['decoded_spans'])
    #metadata = output_res['metadata']
    #print(metadata)

1it [00:00,  8.37it/s]

232b43584b2236669c0a53702ad89ab10c3886ea
[]
23d2d3a6ffebfecaa8930307fdcf451c147757c8


3it [00:00,  7.91it/s]

[]
23d2d3a6ffebfecaa8930307fdcf451c147757c8
[]
23d2d3a6ffebfecaa8930307fdcf451c147757c8


5it [00:00,  8.48it/s]

...

20it [00:02,  9.73it/s]

[]
269730dbbabed8b8b5ba720e44a4c31b1f51e8f1
[]
269730dbbabed8b8b5ba720e44a4c31b1f51e8f1
[]
270e65acc071b9e4e2a632720130c0e10cb6fa08


23it [00:02,  9.44it/s]

[]
270e65acc071b9e4e2a632720130c0e10cb6fa08
[]
270e65acc071b9e4e2a632720130c0e10cb6fa08


KeyboardInterrupt: 

In [ ]:
for batch in tqdm(iterator):
    print(b)
    batch = nn_util.move_to_device(batch, cuda_device)  # Put on GPU.
    output_res = model.decode_saliency(batch, saliency_threshold)

    metadata = output_res['metadata']
    doc_ids: List[str] = [m["doc_id"] for m in metadata]
    assert len(set(doc_ids)) == 1

    decoded_spans: List[Dict[tuple, float]] = output_res['decoded_spans']
    doc_id = metadata[0]['doc_id']

    if doc_id not in documents :
        documents[doc_id] = {}
        documents[doc_id]['saliency'] = []
        documents[doc_id]['doc_id'] = doc_id

    for pspans in decoded_spans :
        for span, prob in pspans.items() :
            documents[doc_id]['saliency'].append([span[0], span[1], 1 if prob > saliency_threshold else 0, prob])


In [ ]:
    with open(output_file, "w") as f:
        documents = {}
        for batch in tqdm(iterator):
            batch = nn_util.move_to_device(batch, cuda_device)  # Put on GPU.
            output_res = model.decode_saliency(batch, saliency_threshold)
            
            metadata = output_res['metadata']
            doc_ids: List[str] = [m["doc_id"] for m in metadata]
            assert len(set(doc_ids)) == 1

            decoded_spans: List[Dict[tuple, float]] = output_res['decoded_spans']
            doc_id = metadata[0]['doc_id']

            if doc_id not in documents :
                documents[doc_id] = {}
                documents[doc_id]['saliency'] = []
                documents[doc_id]['doc_id'] = doc_id

            for pspans in decoded_spans :
                for span, prob in pspans.items() :
                    documents[doc_id]['saliency'].append([span[0], span[1], 1 if prob > saliency_threshold else 0, prob])

        f.write("\n".join([json.dumps(x) for x in documents.values()]))

In [21]:
print(instance['metadata'].metadata)

{'doc_id': '455da02e5048dffb51fb6ab5eb8aeca5926c9d9a', 'paragraph_num': 47, 'paragraph': ['bibliography', ':', 'References', 'section', ':', 'Changelog', 'arXiv', 'v1', '.', 'Initial', 'technical', 'report', 'for', 'ECCV', '2014', 'paper', '.', 'arXiv', 'v2', '.', 'Submitted', 'version', 'for', 'TPAMI', '.', 'Includes', 'extra', 'experiments', 'of', 'SPP', 'on', 'various', 'architectures', '.', 'Includes', 'details', 'for', 'ILSVRC', '2014', '.', 'arXiv', 'v3', '.', 'Accepted', 'version', 'for', 'TPAMI', '.', 'Includes', 'comparisons', 'with', 'R', '-', 'CNN', 'using', 'the', 'same', 'architecture', '.', 'Includes', 'detection', 'experiments', 'using', 'EdgeBoxes', '.', 'arXiv', 'v4', '.', 'Revised', '“', 'Mapping', 'a', 'Window', 'to', 'Feature', 'Maps', '”', 'in', 'Appendix', 'for', 'easier', 'implementation', '.'], 'start_pos_in_doc': 9779, 'end_pos_in_doc': 9862, 'ner_dict': {}, 'sentence_indices': [(9779, 9782), (9782, 9785), (9785, 9788), (9788, 9796), (9796, 9797), (9797, 9799),

In [ ]:

def predict(archive_folder, test_file, output_file, cuda_device):
    '''
    test_file contains atleast - doc_id, sections, sentences, ner in scirex format.

    output_file - {
        'doc_id' : str,
        'saliency' : Tuple[start_index, end_index, salient (binary), saliency probability]
    }
    '''
    import_submodules("scirex")
    logging.info("Loading Model from %s", archive_folder)
    archive_file = os.path.join(archive_folder, "model.tar.gz")
    archive = load_archive(archive_file, cuda_device)
    model = archive.model
    model.eval()

    saliency_threshold = json.load(open(archive_folder + '/metrics.json'))['best_validation__span_threshold']

    model.prediction_mode = True
    config = archive.config.duplicate()
    dataset_reader_params = config["dataset_reader"]
    dataset_reader = DatasetReader.from_params(dataset_reader_params)
    dataset_reader.prediction_mode = True
    instances = dataset_reader.read(test_file)

    for instance in instances :
        batch = Batch([instance])
        batch.index_instances(model.vocab)

    data_iterator = DataIterator.from_params(config["validation_iterator"])
    iterator = data_iterator(instances, num_epochs=1, shuffle=False)

    with open(output_file, "w") as f:
        documents = {}
        for batch in tqdm(iterator):
            batch = nn_util.move_to_device(batch, cuda_device)  # Put on GPU.
            output_res = model.decode_saliency(batch, saliency_threshold)
            
            metadata = output_res['metadata']
            doc_ids: List[str] = [m["doc_id"] for m in metadata]
            assert len(set(doc_ids)) == 1

            decoded_spans: List[Dict[tuple, float]] = output_res['decoded_spans']
            doc_id = metadata[0]['doc_id']

            if doc_id not in documents :
                documents[doc_id] = {}
                documents[doc_id]['saliency'] = []
                documents[doc_id]['doc_id'] = doc_id

            for pspans in decoded_spans :
                for span, prob in pspans.items() :
                    documents[doc_id]['saliency'].append([span[0], span[1], 1 if prob > saliency_threshold else 0, prob])

        f.write("\n".join([json.dumps(x) for x in documents.values()]))





### Reader

In [89]:
import json
from collections import OrderedDict
from typing import Any, Dict, List, Set, Tuple

from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import (
    ListField,
    MetadataField,
    SequenceLabelField,
    SpanField,
    TextField,
)
from scirex.data.dataset_readers.multi_label_field import MultiLabelField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import TokenIndexer
from allennlp.data.tokenizers import Token
from overrides import overrides

from scirex.data.utils.paragraph_alignment import *
from scirex.data.utils.section_feature_extraction import extract_sentence_features
from scirex.data.utils.span_utils import does_overlap, is_x_in_y, spans_to_bio_tags
from scirex_utilities.entity_utils import used_entities

from scipy.stats import mode

Span = Tuple[int, int]  # eg. (0, 10)
ClusterName = str
BaseEntityType = str  # eg. Method
EntityType = Tuple[str, str]  # eg. (Method, True)


In [ ]:
instances = dataset_reader.read(test_file)

In [93]:
file_path=test_file

In [95]:

max_paragraph_length = 300
lazy = False
to_scirex_converter = False

In [ ]:
    type: 'scirex_full_reader',
    token_indexers: token_indexers,

In [ ]:
{'token_indexers': {'bert': {'do_lowercase': 'true', 'pretrained_model': '/cluster/scratch/fgonzalez/scibert/scibert_scivocab_uncased//vocab.txt', 'truncate_long_sequences': False, 'type': 'bert-pretrained', 'use_starting_offsets': True}}

In [132]:
_token_indexers: Dict[str, TokenIndexer] = token_indexers

In [140]:
_token_indexers={"bert": TokenIndexer(token_indexers)}

In [138]:
_token_indexers

In [131]:
token_indexers={"token_indexers" : {
    "bert": {
      "type": "bert-pretrained",
      "pretrained_model": '/cluster/scratch/fgonzalez/scibert/scibert_scivocab_uncased//vocab.txt',
      "do_lowercase": 'true',
      "use_starting_offsets": True,
      "truncate_long_sequences" : False
    }
  }}

In [136]:
token_indexers = {
    "bert": {
      "type": "bert-pretrained",
      "pretrained_model": '/cluster/scratch/fgonzalez/scibert/scibert_scivocab_uncased//vocab.txt',
      "do_lowercase": 'true',
      "use_starting_offsets": True,
      "truncate_long_sequences" : False
    }
  }

In [99]:



_max_paragraph_length = max_paragraph_length
prediction_mode = False

## Hack so I can reuse same reader to convert scirex
## format to scierc format
to_scierc_converter = False

In [119]:
_token_indexers = token_indexers

In [108]:
with open(file_path, "r") as g:
    for _, line in enumerate(g):
        json_dict = json.loads(line)
        if prediction_mode:
            if "method_subrelations" in json_dict:
                del json_dict["method_subrelations"]
            json_dict["n_ary_relations"] = []
        json_dict = clean_json_dict(json_dict)

        verify_json_dict(json_dict)

        # Get fields from JSON dict
        doc_id = json_dict["doc_id"]
        sections: List[Span] = json_dict["sections"]
        sentences: List[List[Span]] = json_dict["sentences"]
        words: List[str] = json_dict["words"]
        entities: Dict[Span, EntityType] = json_dict["ner"]
        corefs: Dict[ClusterName, List[Span]] = json_dict["coref"]
        n_ary_relations: List[Dict[BaseEntityType, ClusterName]] = {}#json_dict["n_ary_relations"]

        # Extract Document structure features
        entities_to_features_map: Dict[Span, List[str]] = extract_sentence_features(
            sentences, words, entities
        )

        # Map cluster names to integer cluster ids
        cluster_name_to_id: Dict[ClusterName, int] = {
            k: i for i, k in enumerate(sorted(list(corefs.keys())))
        }
        max_salient_cluster = len(corefs)

        # Map Spans to list of clusters ids it belong to.
        span_to_cluster_ids: Dict[Span, List[int]] = {}
        for cluster_name in corefs:
            for span in corefs[cluster_name]:
                span_to_cluster_ids.setdefault(span, []).append(cluster_name_to_id[cluster_name])

        span_to_cluster_ids = {span: sorted(v) for span, v in span_to_cluster_ids.items()}

        assert sorted(list(cluster_name_to_id.values())) == list(
            range(max_salient_cluster)
        ), breakpoint()

        # Map types to list of cluster ids that are of that type
        type_to_cluster_ids: Dict[BaseEntityType, List[int]] = {k: [] for k in used_entities}

        for cluster_name in corefs:
            types = [entities[span][0] for span in corefs[cluster_name]]
            if len(set(types)) > 0:
                try :
                    type_to_cluster_ids[mode(types)[0][0]].append(cluster_name_to_id[cluster_name])
                except :
                    # SciERC gives trouble here. Not relevant .
                    continue

        # Map relations to list of cluster ids in it.
        relation_to_cluster_ids: Dict[int, List[int]] = {}
        for rel_idx, rel in enumerate(n_ary_relations):
            relation_to_cluster_ids[rel_idx] = []
            for entity in used_entities:
                relation_to_cluster_ids[rel_idx].append(cluster_name_to_id[rel[entity]])
                type_to_cluster_ids[entity].append(cluster_name_to_id[rel[entity]])

            relation_to_cluster_ids[rel_idx] = tuple(relation_to_cluster_ids[rel_idx])

        for k in type_to_cluster_ids:
            type_to_cluster_ids[k] = sorted(list(set(type_to_cluster_ids[k])))

        # Move paragraph boundaries around to accomodate in BERT
        sections, sentences_grouped, entities_grouped = resize_sections_and_group(
            sections, sentences, entities
        )

        document_metadata = {
            "cluster_name_to_id": cluster_name_to_id,
            "span_to_cluster_ids": span_to_cluster_ids,
            "relation_to_cluster_ids": relation_to_cluster_ids,
            "type_to_cluster_ids": type_to_cluster_ids,
            "doc_id": doc_id,
            "doc_length": len(words),
            "entities_to_features_map": entities_to_features_map,
        }

        # Loop over the sections.
        for (paragraph_num, ((start_ix, end_ix), sentences, ner_dict)) in enumerate(
            zip(sections, sentences_grouped, entities_grouped)
        ):
            paragraph = words[start_ix:end_ix]
            if len(paragraph) == 0:
                breakpoint()
        break


In [189]:
json_dict.keys()

dict_keys(['words', 'sections', 'sentences', 'predicted_ner', 'doc_id', 'ner', 'coref'])

In [115]:
paragraph

['M.S.',
 'degrees',
 'in',
 'computer',
 'science',
 'from',
 'Nanjing',
 'University',
 ',',
 'China',
 ',',
 'in',
 '2001',
 'and',
 '2004',
 ',',
 'respectively',
 ',',
 'and',
 'the',
 'Ph',
 '.',
 'D',
 'degree',
 'from',
 'Deakin',
 'University',
 ',',
 'Australia',
 'in',
 '2008',
 '.',
 'He',
 'joined',
 'the',
 'School',
 'of',
 'Computer',
 'Science',
 'and',
 'Engineering',
 'at',
 'Southeast',
 'University',
 ',',
 'China',
 ',',
 'in',
 '2008',
 ',',
 'and',
 'is',
 'currently',
 'a',
 'professor',
 'and',
 'vice',
 'dean',
 'of',
 'the',
 'school',
 '.',
 'He',
 'has',
 'authored',
 'over',
 '50',
 'refereed',
 'papers',
 ',',
 'and',
 'he',
 'holds',
 'five',
 'patents',
 'in',
 'these',
 'areas',
 '.',
 'His',
 'research',
 'interests',
 'include',
 'pattern',
 'recognition',
 ',',
 'machine',
 'learning',
 ',',
 'and',
 'computer',
 'vision',
 '.']

In [141]:
for indexer_name, indexer in _token_indexers.items():
    print(indexer_name)
    array_keys = indexer.get_keys(indexer_name)

bert


In [125]:
indexer.get_keys()

AttributeError: 'dict' object has no attribute 'get_keys'

In [126]:
_token_indexers.get_keys()

AttributeError: 'dict' object has no attribute 'get_keys'

In [121]:
_token_indexers.items()

dict_items([('bert', {'type': 'bert-pretrained', 'pretrained_model': '/cluster/scratch/fgonzalez/scibert/scibert_scivocab_uncased//vocab.txt', 'do_lowercase': 'true', 'use_starting_offsets': True, 'truncate_long_sequences': False})])

In [142]:
instance = text_to_instance(
                        paragraph_num=paragraph_num,
                        paragraph=paragraph,
                        ner_dict=ner_dict,
                        start_ix=start_ix,
                        end_ix=end_ix,
                        sentence_indices=sentences,
                        document_metadata=document_metadata,
                    )

In [144]:
instance.fields

{'text': <allennlp.data.fields.text_field.TextField at 0x2adc505784d0>,
 'ner_type_labels': <allennlp.data.fields.sequence_label_field.SequenceLabelField at 0x2adc50578890>,
 'metadata': <allennlp.data.fields.metadata_field.MetadataField at 0x2adc505787d0>,
 'spans': <allennlp.data.fields.list_field.ListField at 0x2adc50578210>,
 'span_cluster_labels': <allennlp.data.fields.list_field.ListField at 0x2adc50578b90>,
 'span_saliency_labels': <allennlp.data.fields.sequence_label_field.SequenceLabelField at 0x2adc505781d0>,
 'span_type_labels': <allennlp.data.fields.sequence_label_field.SequenceLabelField at 0x2adc50578410>,
 'span_features': <allennlp.data.fields.list_field.ListField at 0x2adc4fadd650>}

In [166]:
to_scierc_converter

False

In [171]:
paragraph_num=paragraph_num
paragraph=paragraph
ner_dict=ner_dict
start_ix=start_ix
end_ix=end_ix
sentence_indices=sentences
document_metadata=document_metadata

In [175]:
text_field = TextField([Token(word) for word in paragraph], _token_indexers)

In [176]:
metadata_field = MetadataField(
    dict(
        doc_id=document_metadata["doc_id"],
        paragraph_num=paragraph_num,
        paragraph=paragraph,
        start_pos_in_doc=start_ix,
        end_pos_in_doc=end_ix,
        ner_dict=ner_dict,
        sentence_indices=sentence_indices,
        document_metadata=document_metadata,
        num_spans=len(ner_dict),
    )
)

In [182]:
ner_type_labels = spans_to_bio_tags(
    [(k[0] - start_ix, k[1] - start_ix, v[0]) for k, v in ner_dict.items()], len(paragraph)
)

In [184]:
ner_dict

{}

In [183]:
print(ner_type_labels)

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [116]:
def text_to_instance(
        paragraph_num: int,
        paragraph: List[str],
        ner_dict: Dict[Span, str],
        start_ix: int,
        end_ix: int,
        sentence_indices: List[Span],
        document_metadata: Dict[str, Any],
    ):

        if to_scierc_converter:
            return dict(
                paragraph_num=paragraph_num,
                paragraph=paragraph,
                ner_dict=ner_dict,
                start_ix=start_ix,
                end_ix=end_ix,
                sentence_indices=sentence_indices,
                document_metadata=document_metadata,
            )

        text_field = TextField([Token(word) for word in paragraph], _token_indexers)

        metadata_field = MetadataField(
            dict(
                doc_id=document_metadata["doc_id"],
                paragraph_num=paragraph_num,
                paragraph=paragraph,
                start_pos_in_doc=start_ix,
                end_pos_in_doc=end_ix,
                ner_dict=ner_dict,
                sentence_indices=sentence_indices,
                document_metadata=document_metadata,
                num_spans=len(ner_dict),
            )
        )

        ner_type_labels = spans_to_bio_tags(
            [(k[0] - start_ix, k[1] - start_ix, v[0]) for k, v in ner_dict.items()], len(paragraph)
        )

        ner_entity_field = SequenceLabelField(ner_type_labels, text_field, label_namespace="ner_type_labels")

        # Pull it  all together.
        fields = dict(text=text_field, ner_type_labels=ner_entity_field, metadata=metadata_field)

        spans = []
        span_cluster_labels = []
        span_saliency_labels = []
        span_type_labels = []
        span_features = []

        entities_to_features_map = document_metadata["entities_to_features_map"]
        cluster_name_to_id = document_metadata["cluster_name_to_id"]
        relation_to_cluster_ids = document_metadata["relation_to_cluster_ids"]
        span_to_cluster_ids = document_metadata["span_to_cluster_ids"]

        for (s, e), label in ner_dict.items():
            spans.append(SpanField(int(s - start_ix), int(e - start_ix - 1), text_field))
            span_cluster_labels.append(
                MultiLabelField(
                    span_to_cluster_ids.get((s, e), []),
                    label_namespace="cluster_labels",
                    skip_indexing=True,
                    num_labels=len(cluster_name_to_id),
                )
            )
            span_saliency_labels.append(1 if label[-1] == "True" else 0)
            span_type_labels.append(label[0])
            span_features.append(
                MultiLabelField(entities_to_features_map[(s, e)], label_namespace="section_feature_labels", num_labels=5)
            )

        if len(spans) > 0:
            fields["spans"] = ListField(spans)
            fields["span_cluster_labels"] = ListField(span_cluster_labels)
            fields["span_saliency_labels"] = SequenceLabelField(
                span_saliency_labels, fields["spans"], label_namespace="span_saliency_labels"
            )
            fields["span_type_labels"] = SequenceLabelField(
                span_type_labels, fields["spans"], label_namespace="span_type_labels"
            )
            fields["span_features"] = ListField(span_features)
        else:  # Some paragraphs may not have anything !
            fields["spans"] = ListField([SpanField(-1, -1, text_field).empty_field()]).empty_field()
            fields["span_cluster_labels"] = ListField(
                [
                    MultiLabelField(
                        [],
                        label_namespace="cluster_labels",
                        skip_indexing=True,
                        num_labels=len(cluster_name_to_id),
                    )
                ]
            ) #.empty_field()
            fields["span_saliency_labels"] = SequenceLabelField(
                [0], fields["spans"], label_namespace="span_saliency_labels"
            )
            fields["span_type_labels"] = SequenceLabelField(
                ["Method"], fields["spans"], label_namespace="span_type_labels"
            )
            fields["span_features"] = ListField(
                [MultiLabelField([], label_namespace="section_feature_labels", num_labels=5)]
            )

        if len(relation_to_cluster_ids) > 0:
            fields["relation_to_cluster_ids"] = ListField(
                [
                    MultiLabelField(
                        v,
                        label_namespace="cluster_labels",
                        skip_indexing=True,
                        num_labels=len(cluster_name_to_id),
                    )
                    for k, v in relation_to_cluster_ids.items()
                ]
            )

        return Instance(fields)

In [161]:
from allennlp.data.fields.span_field import SpanField

In [162]:
SpanField??

In [165]:
print(instance.get('spans'))

ListField of 1 SpanFields : 
 	 SpanField with spans: (-1, -1). 



In [117]:
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer

In [ ]:
{"tokens": SingleIdTokenIndexer()}

In [101]:
def clean_json_dict(json_dict):
    # Get fields from JSON dict
    entities: List[Tuple[int, int, BaseEntityType]] = json_dict.get("ner", [])
    # Convert Entities to dictionary {(s, e) -> type}
    entities = sorted(entities, key=lambda x: (x[0], x[1]))
    entities: Dict[Span, BaseEntityType] = OrderedDict([((s, e), t) for s, e, t in entities])

    clusters_dict: Dict[ClusterName, List[Span]] = {
        cluster_name: sorted(list(set([tuple(x) for x in spans])))
        for cluster_name, spans in json_dict.get('coref', dict()).items()
    }

    n_ary_relations: List[Dict[BaseEntityType, ClusterName]] = [x for x in json_dict.get("n_ary_relations", list())]
    existing_entities = set([v for relation in n_ary_relations for k, v in relation.items()])

    cluster_to_type: Dict[ClusterName, BaseEntityType] = {}
    for rel in n_ary_relations:
        for k, v in rel.items():
            cluster_to_type[v] = k

    # Under current model, we do not use method subrelations as separate component
    # Therefore, we add each submethod as a separate entity
    if "method_subrelations" in json_dict:
        # Map each method to set containing (all submethod names and the method name itself) .
        method_subrelations: Dict[ClusterName, Set[ClusterName]] = {
            k: set([k] + [x[1] for x in v]) for k, v in json_dict["method_subrelations"].items()
        }

        # Add each submethod to cluster_to_type as Method
        for method_name, method_sub in method_subrelations.items():
            for m in method_sub:
                if m in clusters_dict and m != method_name and m not in existing_entities:
                    clusters_dict[method_name] += clusters_dict[m]
                    clusters_dict[method_name] = sorted(list(set(clusters_dict[method_name])))

                    del clusters_dict[m]

    for cluster, spans in clusters_dict.items():
        for span in spans:
            assert span in entities, breakpoint()
            if cluster not in cluster_to_type:
                continue
            entities[span] = cluster_to_type[cluster]

    for e in entities:
        entities[e]: EntityType = (entities[e], str(any(e in v for v in clusters_dict.values())))

    json_dict["ner"]: Dict[Span, BaseEntityType] = entities
    json_dict["coref"]: Dict[ClusterName, List[Span]] = clusters_dict

    for e in entities:
        in_sentences = [
            i for i, s in enumerate(json_dict["sentences"]) if is_x_in_y(e, s)
        ]  # Check entity lie in one sentence
        if len(in_sentences) > 1:
            breakpoint()
        if len(in_sentences) == 0:
            in_sentences = [i for i, s in enumerate(json_dict["sentences"]) if does_overlap(e, s)]
            assert sorted(in_sentences) == list(range(min(in_sentences), max(in_sentences) + 1)), breakpoint()
            # breakpoint()
            in_sentences = sorted(in_sentences)
            json_dict["sentences"][in_sentences[0]][1] = json_dict["sentences"][in_sentences[-1]][1]
            json_dict["sentences"] = [
                s for i, s in enumerate(json_dict["sentences"]) if i not in in_sentences[1:]
            ]

    json_dict["sentences"]: List[List[Span]] = group_sentences_to_sections(
        json_dict["sentences"], json_dict["sections"]
    )

    return json_dict

In [103]:
def verify_json_dict(json_dict):
    sentences: List[List[Span]] = json_dict["sentences"]
    sections: List[Span] = json_dict["sections"]
    entities: Dict[Span, str] = json_dict["ner"]
    corefs: Dict[str, List[Span]] = json_dict["coref"]

    assert all(sum(is_x_in_y(e, s) for s in sections) == 1 for e in entities), breakpoint()
    assert all(sum(is_x_in_y(e, ss) for s in sentences for ss in s) == 1 for e in entities), breakpoint()
    assert all(
        (sections[i][0] == sentences[i][0][0] and sections[i][-1] == sentences[i][-1][-1])
        for i in range(len(sections))
    ), breakpoint()

    assert all(x in entities for k, v in corefs.items() for x in v), breakpoint()

In [ ]:








@DatasetReader.register("scirex_full_reader")
class ScirexFullReader(DatasetReader):
    def __init__(
        self,
        token_indexers: Dict[str, TokenIndexer] = None,
        max_paragraph_length: int = 300,
        lazy: bool = False,
        to_scirex_converter: bool = False,
    ) -> None:
        super().__init__(lazy)
        self._token_indexers = token_indexers
        self._max_paragraph_length = max_paragraph_length
        self.prediction_mode = False

        ## Hack so I can reuse same reader to convert scirex
        ## format to scierc format
        self.to_scierc_converter = to_scirex_converter

    @overrides
    def _read(self, file_path: str):
        with open(file_path, "r") as g:
            for _, line in enumerate(g):
                json_dict = json.loads(line)
                if self.prediction_mode:
                    if "method_subrelations" in json_dict:
                        del json_dict["method_subrelations"]
                    json_dict["n_ary_relations"] = []
                json_dict = clean_json_dict(json_dict)

                verify_json_dict(json_dict)

                # Get fields from JSON dict
                doc_id = json_dict["doc_id"]
                sections: List[Span] = json_dict["sections"]
                sentences: List[List[Span]] = json_dict["sentences"]
                words: List[str] = json_dict["words"]
                entities: Dict[Span, EntityType] = json_dict["ner"]
                corefs: Dict[ClusterName, List[Span]] = json_dict["coref"]
                n_ary_relations: List[Dict[BaseEntityType, ClusterName]] = json_dict["n_ary_relations"]

                # Extract Document structure features
                entities_to_features_map: Dict[Span, List[str]] = extract_sentence_features(
                    sentences, words, entities
                )

                # Map cluster names to integer cluster ids
                cluster_name_to_id: Dict[ClusterName, int] = {
                    k: i for i, k in enumerate(sorted(list(corefs.keys())))
                }
                max_salient_cluster = len(corefs)

                # Map Spans to list of clusters ids it belong to.
                span_to_cluster_ids: Dict[Span, List[int]] = {}
                for cluster_name in corefs:
                    for span in corefs[cluster_name]:
                        span_to_cluster_ids.setdefault(span, []).append(cluster_name_to_id[cluster_name])

                span_to_cluster_ids = {span: sorted(v) for span, v in span_to_cluster_ids.items()}

                assert sorted(list(cluster_name_to_id.values())) == list(
                    range(max_salient_cluster)
                ), breakpoint()

                # Map types to list of cluster ids that are of that type
                type_to_cluster_ids: Dict[BaseEntityType, List[int]] = {k: [] for k in used_entities}

                for cluster_name in corefs:
                    types = [entities[span][0] for span in corefs[cluster_name]]
                    if len(set(types)) > 0:
                        try :
                            type_to_cluster_ids[mode(types)[0][0]].append(cluster_name_to_id[cluster_name])
                        except :
                            # SciERC gives trouble here. Not relevant .
                            continue

                # Map relations to list of cluster ids in it.
                relation_to_cluster_ids: Dict[int, List[int]] = {}
                for rel_idx, rel in enumerate(n_ary_relations):
                    relation_to_cluster_ids[rel_idx] = []
                    for entity in used_entities:
                        relation_to_cluster_ids[rel_idx].append(cluster_name_to_id[rel[entity]])
                        type_to_cluster_ids[entity].append(cluster_name_to_id[rel[entity]])

                    relation_to_cluster_ids[rel_idx] = tuple(relation_to_cluster_ids[rel_idx])

                for k in type_to_cluster_ids:
                    type_to_cluster_ids[k] = sorted(list(set(type_to_cluster_ids[k])))

                # Move paragraph boundaries around to accomodate in BERT
                sections, sentences_grouped, entities_grouped = self.resize_sections_and_group(
                    sections, sentences, entities
                )

                document_metadata = {
                    "cluster_name_to_id": cluster_name_to_id,
                    "span_to_cluster_ids": span_to_cluster_ids,
                    "relation_to_cluster_ids": relation_to_cluster_ids,
                    "type_to_cluster_ids": type_to_cluster_ids,
                    "doc_id": doc_id,
                    "doc_length": len(words),
                    "entities_to_features_map": entities_to_features_map,
                }

                # Loop over the sections.
                for (paragraph_num, ((start_ix, end_ix), sentences, ner_dict)) in enumerate(
                    zip(sections, sentences_grouped, entities_grouped)
                ):
                    paragraph = words[start_ix:end_ix]
                    if len(paragraph) == 0:
                        breakpoint()

                    instance = self.text_to_instance(
                        paragraph_num=paragraph_num,
                        paragraph=paragraph,
                        ner_dict=ner_dict,
                        start_ix=start_ix,
                        end_ix=end_ix,
                        sentence_indices=sentences,
                        document_metadata=document_metadata,
                    )
                    yield instance




In [107]:
def resize_sections_and_group(
     sections: List[Span], sentences: List[List[Span]], entities: Dict[Span, EntityType]
):
    broken_sections = move_boundaries(
        break_paragraphs(
            collapse_paragraphs(sections, min_len=20, max_len=_max_paragraph_length),
            max_len=_max_paragraph_length,
        ),
        list(entities.keys()),
    )

    for p, q in zip(broken_sections[:-1], broken_sections[1:]):
        if p[1] != q[0] or p[1] < p[0] or q[1] < q[0]:
            breakpoint()

    sections = broken_sections
    entities_grouped = [{} for _ in range(len(sections))]
    sentences_grouped = [[] for _ in range(len(sections))]

    # Bert is PITA. Group entities into sections they belong to.
    for e in entities:
        is_in_n_para = 0
        for para_id, p in enumerate(sections):
            if is_x_in_y(e, p):
                entities_grouped[para_id][(e[0], e[1])] = entities[e]
                is_in_n_para += 1

        assert is_in_n_para == 1, breakpoint()

    ## Bert is serious PITA. Need to align sentences with sections also.
    sentences = [sent for section in sentences for sent in section]
    assert all([sentences[i + 1][0] == sentences[i][1] for i in range(len(sentences) - 1)]), breakpoint()
    assert sentences[-1][1] == sections[-1][1], breakpoint()
    sentence_indices = sorted(list(set([0] + [s[1] for s in sentences] + [s[1] for s in sections])))
    sentences = list(zip(sentence_indices[:-1], sentence_indices[1:]))
    for e in sentences:
        is_in_n_para = 0
        for para_id, p in enumerate(sections):
            if is_x_in_y(e, p):
                sentences_grouped[para_id].append(e)
                is_in_n_para += 1

        assert is_in_n_para == 1, breakpoint()

    zipped = zip(sections, sentences_grouped, entities_grouped)

    # Remove Empty sections
    sections, sentences_grouped, entities_grouped = [], [], []
    for p, s, e in zipped:
        if p[1] - p[0] == 0:
            assert len(e) == 0, breakpoint()
            assert len(s) == 0, breakpoint()
            continue
        sections.append(p)
        entities_grouped.append(e)
        sentences_grouped.append(s)

    return sections, sentences_grouped, entities_grouped

In [ ]:


    def text_to_instance(
        paragraph_num: int,
        paragraph: List[str],
        ner_dict: Dict[Span, str],
        start_ix: int,
        end_ix: int,
        sentence_indices: List[Span],
        document_metadata: Dict[str, Any],
    ):

        if to_scierc_converter:
            return dict(
                paragraph_num=paragraph_num,
                paragraph=paragraph,
                ner_dict=ner_dict,
                start_ix=start_ix,
                end_ix=end_ix,
                sentence_indices=sentence_indices,
                document_metadata=document_metadata,
            )

        text_field = TextField([Token(word) for word in paragraph], _token_indexers)

        metadata_field = MetadataField(
            dict(
                doc_id=document_metadata["doc_id"],
                paragraph_num=paragraph_num,
                paragraph=paragraph,
                start_pos_in_doc=start_ix,
                end_pos_in_doc=end_ix,
                ner_dict=ner_dict,
                sentence_indices=sentence_indices,
                document_metadata=document_metadata,
                num_spans=len(ner_dict),
            )
        )

        ner_type_labels = spans_to_bio_tags(
            [(k[0] - start_ix, k[1] - start_ix, v[0]) for k, v in ner_dict.items()], len(paragraph)
        )

        ner_entity_field = SequenceLabelField(ner_type_labels, text_field, label_namespace="ner_type_labels")

        # Pull it  all together.
        fields = dict(text=text_field, ner_type_labels=ner_entity_field, metadata=metadata_field)

        spans = []
        span_cluster_labels = []
        span_saliency_labels = []
        span_type_labels = []
        span_features = []

        entities_to_features_map = document_metadata["entities_to_features_map"]
        cluster_name_to_id = document_metadata["cluster_name_to_id"]
        relation_to_cluster_ids = document_metadata["relation_to_cluster_ids"]
        span_to_cluster_ids = document_metadata["span_to_cluster_ids"]

        for (s, e), label in ner_dict.items():
            spans.append(SpanField(int(s - start_ix), int(e - start_ix - 1), text_field))
            span_cluster_labels.append(
                MultiLabelField(
                    span_to_cluster_ids.get((s, e), []),
                    label_namespace="cluster_labels",
                    skip_indexing=True,
                    num_labels=len(cluster_name_to_id),
                )
            )
            span_saliency_labels.append(1 if label[-1] == "True" else 0)
            span_type_labels.append(label[0])
            span_features.append(
                MultiLabelField(entities_to_features_map[(s, e)], label_namespace="section_feature_labels", num_labels=5)
            )

        if len(spans) > 0:
            fields["spans"] = ListField(spans)
            fields["span_cluster_labels"] = ListField(span_cluster_labels)
            fields["span_saliency_labels"] = SequenceLabelField(
                span_saliency_labels, fields["spans"], label_namespace="span_saliency_labels"
            )
            fields["span_type_labels"] = SequenceLabelField(
                span_type_labels, fields["spans"], label_namespace="span_type_labels"
            )
            fields["span_features"] = ListField(span_features)
        else:  # Some paragraphs may not have anything !
            fields["spans"] = ListField([SpanField(-1, -1, text_field).empty_field()]).empty_field()
            fields["span_cluster_labels"] = ListField(
                [
                    MultiLabelField(
                        [],
                        label_namespace="cluster_labels",
                        skip_indexing=True,
                        num_labels=len(cluster_name_to_id),
                    )
                ]
            ) #.empty_field()
            fields["span_saliency_labels"] = SequenceLabelField(
                [0], fields["spans"], label_namespace="span_saliency_labels"
            )
            fields["span_type_labels"] = SequenceLabelField(
                ["Method"], fields["spans"], label_namespace="span_type_labels"
            )
            fields["span_features"] = ListField(
                [MultiLabelField([], label_namespace="section_feature_labels", num_labels=5)]
            )

        if len(relation_to_cluster_ids) > 0:
            fields["relation_to_cluster_ids"] = ListField(
                [
                    MultiLabelField(
                        v,
                        label_namespace="cluster_labels",
                        skip_indexing=True,
                        num_labels=len(cluster_name_to_id),
                    )
                    for k, v in relation_to_cluster_ids.items()
                ]
            )

        return Instance(fields)

In [192]:
#! /usr/bin/env python

import json
import os
from sys import argv
from typing import Dict, List

from tqdm import tqdm

from allennlp.common.util import import_submodules
from allennlp.data import DataIterator, DatasetReader
from allennlp.data.dataset import Batch
from allennlp.models.archival import load_archive
from allennlp.nn import util as nn_util
from allennlp.common.params import Params

import logging
logging.basicConfig(format="%(asctime)s:%(levelname)s:%(message)s", level=logging.INFO)


def predict(archive_folder, test_file, output_file, cuda_device):
    '''
    test_file contains atleast - doc_id, sections, sentences, ner in scirex format.

    output_file - {
        'doc_id' : str,
        'saliency' : Tuple[start_index, end_index, salient (binary), saliency probability]
    }
    '''
    import_submodules("scirex")
    logging.info("Loading Model from %s", archive_folder)
    archive_file = os.path.join(archive_folder, "model.tar.gz")
    archive = load_archive(archive_file, cuda_device)
    model = archive.model
    model.eval()

    saliency_threshold = json.load(open(archive_folder + '/metrics.json'))['best_validation__span_threshold']

    model.prediction_mode = True
    config = archive.config.duplicate()
    dataset_reader_params = config["dataset_reader"]
    dataset_reader = DatasetReader.from_params(dataset_reader_params)
    dataset_reader.prediction_mode = True
    instances = dataset_reader.read(test_file)

    for instance in instances :
        batch = Batch([instance])
        batch.index_instances(model.vocab)
    data_iterator = DataIterator.from_params(config["validation_iterator"])
    iterator = data_iterator(instances, num_epochs=1, shuffle=False)

    with open(output_file, "w") as f:
        documents = {}
        for batch in tqdm(iterator):
            batch = nn_util.move_to_device(batch, cuda_device)  # Put on GPU.
            output_res = model.decode_saliency(batch, saliency_threshold)
            if "metadata" not in output_res:
                continue
            metadata = output_res['metadata']
            doc_ids: List[str]= []
            for m in metadata:
                if "doc_id" in m:
                    doc_ids.append(m["doc_id"])
            if len(doc_ids) == 0:
                continue

            assert len(set(doc_ids)) == 1
            
            #doc_ids: List[str] = [m["doc_id"] for m in metadata]
            #assert len(set(doc_ids)) == 1

            decoded_spans: List[Dict[tuple, float]] = output_res['decoded_spans']
            doc_id = metadata[0]['doc_id']

            if doc_id not in documents :
                documents[doc_id] = {}
                documents[doc_id]['saliency'] = []
                documents[doc_id]['doc_id'] = doc_id

            for pspans in decoded_spans :
                for span, prob in pspans.items() :
                    documents[doc_id]['saliency'].append([span[0], span[1], 1 if prob > saliency_threshold else 0, prob])

        f.write("\n".join([json.dumps(x) for x in documents.values()]))


def main():
    archive_folder = ""#argv[1]
    test_file = ""#argv[2]
    output_file = ""#argv[3]
    cuda_device = ""#int(argv[4])
    predict(archive_folder, test_file, output_file, cuda_device)


if __name__ == "__main__":
    main()


2022-03-17 12:32:57,096:INFO:Loading Model from 


FileNotFoundError: file model.tar.gz not found